In [1]:
import pandas as pd
import model_selection_pipeline as pipeline
from joblib import dump
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier, QuantileDMatrix, Booster, train
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.model_selection import GridSearchCV
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

In [2]:
df = pd.read_csv('./datasets/train_cleaned.csv')
df.shape

(100000, 28)

In [4]:
# feature_list = ['Num_of_Delayed_Payment', 'Num_Bank_Accounts', 'Interest_Rate','Delay_from_due_date','Changed_Credit_Limit','Outstanding_Debt','Credit_History_Age', 'Credit_Score']

feature_list = ['Monthly_Inhand_Salary','Annual_Income', 'Amount_invested_monthly','Num_of_Delayed_Payment', 'Num_Bank_Accounts', 'Interest_Rate','Delay_from_due_date','Changed_Credit_Limit','Outstanding_Debt','Credit_History_Age', 'Credit_Score']

In [5]:
subset_df = df[feature_list]

subset_df['Credit_Score'] = subset_df['Credit_Score'].map({'Good': 0, 'Poor': 1, 'Standard': 2})

X = subset_df[feature_list[:-1]].values
y = subset_df[["Credit_Score"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify = y, test_size = 0.3)

C:\Users\aarya\AppData\Local\Temp\ipykernel_32340\3279624015.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['Credit_Score'] = subset_df['Credit_Score'].map({'Good': 0, 'Poor': 1, 'Standard': 2})


In [6]:
dict(Counter(y_train["Credit_Score"]))

{2: 37222, 1: 20299, 0: 12479}

In [7]:
strategy = {1:25000}
sampler = SMOTE(sampling_strategy=strategy)

X_train_new, y_train_new = sampler.fit_resample(X_train, y_train)

In [8]:
y_train_new["Credit_Score"].value_counts()

Credit_Score
2    37222
1    25000
0    12479
Name: count, dtype: int64

In [9]:
params = {
 'base_score': 0.5,
 'colsample_bylevel': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'learning_rate': 1e-1,
 'max_delta_step': 1,
 'max_depth': 15,
 'min_child_weight': 1,
 'n_estimators': 350,
 'objective': 'binary:logistic',
 'reg_alpha': 9e-1,
 'reg_lambda': 0.8,
#  'scale_pos_weight': 1,
 'seed': 21,
#  'silent': 1,
 'subsample': 1,
}

In [10]:
over_sampled_model = XGBClassifier(**params)
over_sampled_model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster=None, callbacks=None, colsample_bylevel=1,
              colsample_bynode=None, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None, max_delta_step=1,
              max_depth=15, max_leaves=None, min_child_weight=1, missing=nan,
              monotone_constraints=None, n_estimators=350, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob',
              predictor=None, ...)

In [11]:
y_pred= over_sampled_model.predict(X_test)

In [12]:
scores = {
    "F1:": "{:.4f}".format(f1_score(y_test, y_pred, average='weighted')),
    "Precision:": "{:.4f}".format(precision_score(y_test, y_pred, average='weighted')),
    "Recall:": "{:.4f}".format(recall_score(y_test, y_pred, average='weighted')),
    "Accuracy:": "{:.4f}".format(accuracy_score(y_test, y_pred))
}
print(scores)

{'F1:': '0.8055', 'Precision:': '0.8059', 'Recall:': '0.8056', 'Accuracy:': '0.8056'}
